# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686526


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:14,  2.65s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:03,  2.34s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:44,  1.72s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:32,  1.29s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:00,  4.12it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  4.64it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  5.41it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  6.05it/s]

zac8151                               0.000293
Carteige                              0.008396
jammiedodgers                         0.001872
lucyxnorth                            0.017698
ctiborekskutr                         0.029498
not-logged-in-975ec4b89e5ce0be836b    0.000432
not-logged-in-f87bd5afe536165d6b05    0.065481
djswanso                              0.000181
thedarktree                           0.003470
mr_willock                            0.000153
not-logged-in-6ead6661c7dcef165774    0.005104
ferlin1998                            0.000574
Jack_07                               0.000526
not-logged-in-5f30b2bb9b7f9d49da41    0.072646
machine9898                           0.002765
jmartinez468                          0.023889
sepsamadi                             0.029023
41Letters                             0.000504
aidenr                                0.000531
acapirala                             0.002199
Lavadude                              0.055447
ShirleyL     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())